In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 11 13:42:27 2023

@author: Zhang Jun
"""

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("data/Originalscreen_Alltissues_fitteddata.csv", header = 0, quotechar = '"')

C:\Users\Zhang Jun\AppData\Local\Temp\ipykernel_13996\2805218171.py:1: DtypeWarning: Columns (6,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/Originalscreen_Alltissues_fitteddata.csv", header = 0, quotechar = '"')


In [3]:
synergy_data = data[data["Synergy"] == True]
anta_data = data[data["Synergy"] == False]

In [4]:
#Get unique "combination-cell line" pairs.
synergy_pair = synergy_data[np.logical_not(synergy_data.duplicated(subset = ["COMBI_ID", "CELL_LINE_NAME"], keep = 'first'))]
anta_pair = anta_data[np.logical_not(anta_data.duplicated(subset = ["COMBI_ID", "CELL_LINE_NAME"], keep = 'first'))]
print(len(synergy_pair), len(anta_pair))

6158 106842


In [5]:
#Get "combination-cell line" pairs with fuzzy labels.
seta = synergy_pair["COMBI_ID"] + "+" + synergy_pair["CELL_LINE_NAME"]
setb = anta_pair["COMBI_ID"] + "+" +anta_pair["CELL_LINE_NAME"]
fuzzy = set(seta) & set(setb)
print(len(fuzzy))

4741


In [ ]:
#Label fuzzy pairs.
for e in fuzzy:
    combi_id = e.split("+")[0]
    cell = e.split("+")[1]
    
    measures_df = data[np.logical_and(data["COMBI_ID"] == combi_id, data["CELL_LINE_NAME"] == cell)]

    if len(pd.unique(measures_df["ANCHOR_CONC"])) == 1:
        if len(measures_df[measures_df["Synergy"] == True]) >= len(measures_df) / 2:
            data.iloc[measures_df.index, data.columns.get_loc('Synergy')] = True
        else:
            data.iloc[measures_df.index, data.columns.get_loc('Synergy')] = False
    #elif len(pd.unique(measures_df["ANCHOR_CONC"])) > 1:
     #   data.iloc[measures_df.index, data.columns.get_loc('Synergy')] = True

In [7]:
combs = pd.DataFrame(columns = data.columns)
for e in fuzzy:
    combi_id = e.split("+")[0]
    cell = e.split("+")[1]
    
    measures_df = data[np.logical_and(data["COMBI_ID"] == combi_id, data["CELL_LINE_NAME"] == cell)]
    combs = pd.concat((combs, measures_df), axis = 0)
    
combs.to_csv("./data/pairwith2labels.csv", header = True, index = True, quotechar = '"')

C:\Users\Zhang Jun\AppData\Local\Temp\ipykernel_13996\3247066373.py:7: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  combs = pd.concat((combs, measures_df), axis = 0)
C:\Users\Zhang Jun\AppData\Local\Temp\ipykernel_13996\3247066373.py:7: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  combs = pd.concat((combs, measures_df), axis = 0)
C:\Users\Zhang Jun\AppData\Local\Temp\ipykernel_13996\3247066373.py:7: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  combs = pd.concat((combs, measures_df), axis = 0)
C:\Users\Zhang Jun\AppData\Local\Temp\ipykernel_13996\3247066373.py:7: FutureWarning: In a future version, objec

In [ ]:
b

In [6]:
#Label fuzzy pairs.
for e in fuzzy:
    combi_id = e.split("+")[0]
    cell = e.split("+")[1]
    
    measures_df = data[np.logical_and(data["COMBI_ID"] == combi_id, data["CELL_LINE_NAME"] == cell)]

    if len(measures_df[measures_df["Synergy"] == True]) >= (len(measures_df) / 2):
        data.iloc[measures_df.index, data.columns.get_loc('Synergy')] = True
    else:
        data.iloc[measures_df.index, data.columns.get_loc('Synergy')] = False
    #elif len(pd.unique(measures_df["ANCHOR_CONC"])) > 1:
     #   data.iloc[measures_df.index, data.columns.get_loc('Synergy')] = True

In [12]:
data.to_csv("./data/data_no_fuzzy.csv", header = True, index = False, quotechar = '"')
synergy_pair.to_csv("./data/synergy_pair_dnf.csv", header = True, index = False, quotechar = '"')
anta_pair.to_csv("./data/anta_pair_dnf.csv", header = True, index = False, quotechar = '"')  